In [1]:
# input
n_topics = 10

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# from tw_dataset.dbmodels import *
from random import sample
import json

In [4]:
from tw_dataset.dbmodels import *

Switching to API Credentials #7


In [5]:
s = open_session()
all_tweets_es = [t for t in s.query(Tweet).all() if t.lang == 'es']
all_tweets_text_es = [t.text for t in all_tweets_es]
all_tweets_es_ids = [t.id for t in all_tweets_es]


In [6]:
len(all_tweets_text_es)    

109040

In [7]:
tweets = all_tweets_text_es

# OK, let's finally do some topic modelling

In [8]:
dictionary = gensim.corpora.Dictionary.load("../tweets_es.dict")

with open('../tweets_es_bow.pickle', 'rb') as f:
    bow = pickle.load(f)

In [14]:
dictionary.items()[:10]

[(0, u'moreauleopold'),
 (1, u'nachnop_mjoliv'),
 (2, u'lacocinadlmied'),
 (3, u'insolit'),
 (4, u'acusticazonacional'),
 (5, u'audienci_public'),
 (6, u'ciudadel'),
 (7, u'indignacion'),
 (8, u'hector_daer'),
 (9, u'hermand')]

In [6]:
for i, t in dictionary.items()[:20]:
    print "%d: %s" % (i, t)

0: moreauleopold
1: nachnop_mjoliv
2: lacocinadlmied
3: insolit
4: acusticazonacional
5: audienci_public
6: ciudadel
7: indignacion
8: hector_daer
9: hermand
10: caus_inund
11: canel
12: soni_aless
13: gmilm
14: tom_decision
15: convocatori_evalu
16: medicament
17: monsenor
18: elgaf
19: elrastaok


In [16]:
phrases = [x for x in dictionary.items() if '_' in x[1]]

In [19]:
phrases[:50]

[(1, u'nachnop_mjoliv'),
 (5, u'audienci_public'),
 (8, u'hector_daer'),
 (10, u'caus_inund'),
 (12, u'soni_aless'),
 (14, u'tom_decision'),
 (15, u'convocatori_evalu'),
 (32, u'cicl_cin'),
 (55, u'ministr_segur'),
 (57, u'inici_juici'),
 (59, u'mendoz_efect'),
 (70, u'nin_nin'),
 (72, u'millon_judi'),
 (74, u'indi_solari'),
 (81, u'mastardeimp_\xa1quedat'),
 (82, u'conden_anos'),
 (85, u'si_cres'),
 (102, u'menor_anos'),
 (107, u'reflexion_caus'),
 (115, u'curr_ddhh'),
 (121, u'dan_clas'),
 (152, u'habi_hech'),
 (163, u'estamosd_radi'),
 (164, u'daniel_scioli'),
 (168, u'desp_indec'),
 (170, u'pagin_web'),
 (172, u'martintetaz_alfiemart'),
 (173, u'twets_retwitt'),
 (180, u'canciller_mercosur'),
 (182, u'via_agenciaelvigi'),
 (184, u'hij_hij'),
 (189, u'hac_rat'),
 (190, u'ex_pte'),
 (192, u'cn_navarroencn'),
 (194, u'gast_plat'),
 (208, u'\xbfpod_ayud'),
 (210, u'vill_urquiz'),
 (221, u'represion_policial'),
 (222, u'felic_pascu'),
 (232, u'litr_agu'),
 (233, u'acto_homenaj'),
 (238,

In [9]:
tweets_rp_inds = [i for i, x in enumerate(bow) if 221 in [ind for (ind,c) in x]]

In [10]:
tweets_rp = [(i, all_tweets_text_es[i]) for i in tweets_rp_inds]

In [11]:
tweets_rp[:50]

[(2326,
  u'Investigar\xe1n la represi\xf3n policial del 8M. @MPFCABA @FiscalesGobAr #NiunaMenos #Vivasnosqueremos\u2026 https://t.co/AX20NkveUw'),
 (17152,
  u'URGENTE: Fuerte represi\xf3n policial en La Boca ante una protesta de vecinos. (\U0001f4f9: @diariofuturo) https://t.co/Qgv9w2UAbQ'),
 (17155,
  u'La Coordinadora contra la Represi\xf3n Policial e Institucional confirma la muerte de una vecina de La Boca. https://t.co/poXpCXoOEM'),
 (17203,
  u'Represi\xf3n policial en Bs. AS. La Boca sitiada. Esta noche. Hay heridos graves. Difundir. Contacto. Luciano: 1163972\u2026 https://t.co/T0HyDPQ6Dp'),
 (17647,
  u'La Boca: una mujer muerta y represi\xf3n policial https://t.co/TUPV86xPst https://t.co/6M9vDThX8H'),
 (19865,
  u'La PutySe\xf1al una APP contra la violencia institucional, para bajarla en el celular y frenar la represi\xf3n policial. https://t.co/sJxmSVBXaK'),
 (21048,
  u'[IMPACTANTE] Represi\xf3n policial en Ministerio de trabajo a empleados de Call Center Task Solution (T

In [12]:
tweet =  all_tweets_text_es[75943]

In [13]:
print tweet

Denuncian que cocinera del comedor de Lanús perdió su embarazo debido a represión policial https://t.co/5Mh8FpFmy7


In [7]:
print [dictionary[i] for (i,c) in bow[75943]]

[u'represion_policial', u'cociner_comedor', u'denunci', u'deb', u'perdi_embaraz', u'lanus']


In [8]:
bow[75943]

[(221, 1), (4789, 1), (6205, 1), (10694, 1), (12627, 1), (22029, 1)]

In [14]:
ind = 90172
tweet = all_tweets_text_es[ind]

In [15]:
print tweet

#HaceInstantes Abrazo al Congreso organizado por gremios docentes después de la represión policial de ayer. https://t.co/ToyertpI70


In [13]:
print [dictionary[i] for (i,c) in bow[ind]]

[u'represion_policial', u'gremi_docent', u'ayer', u'organiz', u'abraz_congres', u'despu']


In [14]:
print bow[ind]

[(221, 1), (1176, 1), (3055, 1), (6257, 1), (14203, 1), (25141, 1)]


In [40]:
ind = 70431

In [41]:
print all_tweets_text_es[ind]

#Urgente: El MTE informa que la cocinera del comedor Laura Zaracho perdio su embarazo como consecuencia de la represión policial.


In [42]:
[dictionary[i] for (i,c) in bow[ind]]

[u'represion_policial',
 u'cociner_comedor',
 u'inform',
 u'laur_zarach',
 u'perdi_embaraz',
 u'consecuent',
 u'mte',
 u'urgent']

In [ ]:
trigram[tokenize(preprocess(tweet), remove_stopwords=True, stem=True)]

In [24]:
tweets_rp_inds

[2326,
 17152,
 17155,
 17203,
 17647,
 19865,
 21048,
 35203,
 48874,
 48906,
 48994,
 49016,
 49102,
 49231,
 49778,
 51492,
 52604,
 68106,
 70090,
 70135,
 70258,
 70431,
 73307,
 75943,
 85490,
 85496,
 85548,
 85647,
 85685,
 85831,
 86033,
 87298,
 90172,
 91163,
 102616]

In [22]:
tweets_rp

[[(221, 1), (16799, 1), (19559, 1), (24794, 1), (25649, 1)],
 [(33, 1), (221, 1), (3708, 1), (9755, 1), (21124, 1), (22308, 1)],
 [(221, 1), (3121, 1), (3741, 1), (7321, 1), (7847, 1), (17122, 1)],
 [(221, 1),
  (3708, 1),
  (4790, 1),
  (6741, 1),
  (8576, 1),
  (8946, 1),
  (13819, 1),
  (15026, 1),
  (16153, 1),
  (17053, 1)],
 [(221, 1), (3708, 1), (5131, 1), (7321, 1)],
 [(221, 1), (5333, 1), (12201, 1), (22558, 1), (23056, 1), (24691, 1)],
 [(221, 1),
  (4383, 1),
  (6610, 1),
  (7510, 1),
  (10229, 1),
  (12490, 1),
  (19741, 1),
  (22287, 1),
  (23905, 1)],
 [(221, 1), (2468, 1), (5552, 1), (12594, 1), (23915, 1)],
 [(221, 1), (6205, 1), (10850, 1), (13493, 1), (22029, 1), (23091, 1)],
 [(221, 1),
  (3785, 1),
  (7555, 1),
  (7704, 1),
  (10525, 1),
  (13084, 1),
  (20719, 1),
  (20910, 1),
  (21523, 1)],
 [(221, 1),
  (2652, 1),
  (6026, 1),
  (17195, 1),
  (19174, 1),
  (22029, 1),
  (23584, 1),
  (24736, 1)],
 [(221, 1),
  (2082, 1),
  (11313, 1),
  (12269, 1),
  (22029, 1),

In [13]:
dictionary[17]

u'monsenor'

In [9]:
dictionary.id2token.items()[:10]

[]

In [17]:
len(phrases)

5156